# Tutorial for ETL pipeline
This tutorial will run you through the basics of ETL using the Riot Games League of Legends API. 

## What is ETL?
ETL stands for **Extract, Transform, Load**. One of the very first steps of any data-related task is to consolidate all your data in a readable and accessible way. The data usually starts out noisy and jumbled and may as well come from many different sources. The natural first step would be to **extract** all the data from these sources. The next step would be to **transform** the data into a concise, usually singular, format that is consisent across its entirety. The last step is to **load** the data into some form of data storage. ETL is a key concept in data science and data engineering as we aim to make more informed decisions about real-life problems. With the ever-expanding sources of data that are available, it is vital that we have a solid methology in prepping the data for whatever task lies ahead.

## Riot Games API
Let's get started with an actual hands-on application of ETL. To do this we are using the [Riot Games API](https://developer.riotgames.com/) for the game League of Legends. If you are unfamiliar with the game, please read the below header to get a brief understanding which will be helpful when looking at the data. If you are familiar, feel free to skip the below paragraph.
### League of Legends
League of Legends is a free-to-play MOBA. Two teams of five are split up across a map where they have to simultaneously attack the enemy side while defending their own. Every person in the game controls a different **champion** with different abilities and playstyles. In order to defeat the enemy team, players acquire **items** which increase champions' power. A team wins by pushing through to the enemy base and destroying their nexus. We will mainly be focusing on the champions and items when creating this ETL pipeline.



To get started with the API, you have to retrieve an API key from the Riot developer portal. I have created an account for anyone using this tutorial, so please login [here](https://auth.riotgames.com/login#redirect_uri=https%3A%2F%2Fdeveloper.riotgames.com%2Foauth2-callback&client_id=riot-developer-portal&response_type=code&scope=openid%20email%20summoner) using the following credentials:

```
Username: 15388tutorial
Password: tutorial123
```

You should be redirected to the developer portal where you can then pass the reCAPTCHA check and click "Regenerate API key". Please copy the key into the code block below. *It should be noted that this key will expire every 24 hours, so if you do the tutorial over a long period of time remember to regenerate your key.*

In [206]:
API_key = "RGAPI-c7ec2289-4873-49f6-a31e-84ebb4eec98a"

# Extract
For the first part of our ETL pipeline, we have to extract the data. The API provides us with all the data in json format. You can get a more in-depth look at the API [here](https://developer.riotgames.com/docs/lol). Let's run a quick extraction to get an idea of what we are working with.  

In [207]:
import requests
link = "https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/Hoque?api_key=" + API_key
print(requests.get(link).text)

{"id":"j44BHU838i-Wai6PpRIRW1dvd1ZBOAGrXibA33dli6WRUrI","accountId":"AzeVInz194AduFR6gDba2USPVWcQxlP_10AgMSMu4CUB3PY","puuid":"p2C3DOljTs4KxKzpwApKU70Dxb3v0IOS3_VDdLlwiOYGsw-lVxcDhDE8dXSV-wYj43_tC3IQQgRsQw","name":"Hoque","profileIconId":684,"revisionDate":1617658576000,"summonerLevel":137}


This simple request uses the API key and some keywords to get specific information. In this case, I accessed the summoner (*a summoner is simply a specific player*) data using my summoner name. It returns a simple json with my unique ID, name, level, and other information specific to my account. Now that you have an idea of how the API functions, we can look at some more data. However accesing it in this format is clearly not very intuitive. Luckily, we have wrappers.

## Cassiopeia
A wrapper is simply a language-specific library for a certain API which makes calls to the API much neater and easier to use. Since we are using python, we will be using a python-specific wrapper called [Cassiopeia](https://cassiopeia.readthedocs.io/en/latest/). If you wish to go further with this tutorial, make sure to read the docs thoroughly! Let's start with installing cassiopeia. Run the following command in terminal:

```
pip install cassiopeia
```

Now that it's installed, let's do some actual extraction. 

In [208]:
import cassiopeia as cass
cass.set_riot_api_key(API_key) 
cass.set_default_region("NA")
champions = cass.get_champions()
print(champions[:10])

[<cassiopeia.core.staticdata.champion.Champion object at 0x11a4753d0>, <cassiopeia.core.staticdata.champion.Champion object at 0x11ae0aa90>, <cassiopeia.core.staticdata.champion.Champion object at 0x11ae0ac10>, <cassiopeia.core.staticdata.champion.Champion object at 0x11ae0ad90>, <cassiopeia.core.staticdata.champion.Champion object at 0x11ae0af10>, <cassiopeia.core.staticdata.champion.Champion object at 0x11cb360d0>, <cassiopeia.core.staticdata.champion.Champion object at 0x11cb36250>, <cassiopeia.core.staticdata.champion.Champion object at 0x11cb363d0>, <cassiopeia.core.staticdata.champion.Champion object at 0x11cb36550>, <cassiopeia.core.staticdata.champion.Champion object at 0x11cb366d0>]


As you can see by using a wrapper, extraction is much easier. The json format the API usually returns is parsed by the wrapper and given in a readable format. For this tutorial, we will just be focusing on the Champions as well as some items although there is plenty of more data you can retrieve using the API. The data is still given in cassiopeia objects, so to get some real information we need to simply use methods defined by the cassiopeia docs.

In [209]:
names = []
for i in champions:
    names.append(i.name)
print(names)

['Aatrox', 'Ahri', 'Akali', 'Alistar', 'Amumu', 'Anivia', 'Annie', 'Aphelios', 'Ashe', 'Aurelion Sol', 'Azir', 'Bard', 'Blitzcrank', 'Brand', 'Braum', 'Caitlyn', 'Camille', 'Cassiopeia', "Cho'Gath", 'Corki', 'Darius', 'Diana', 'Draven', 'Dr. Mundo', 'Ekko', 'Elise', 'Evelynn', 'Ezreal', 'Fiddlesticks', 'Fiora', 'Fizz', 'Galio', 'Gangplank', 'Garen', 'Gnar', 'Gragas', 'Graves', 'Hecarim', 'Heimerdinger', 'Illaoi', 'Irelia', 'Ivern', 'Janna', 'Jarvan IV', 'Jax', 'Jayce', 'Jhin', 'Jinx', "Kai'Sa", 'Kalista', 'Karma', 'Karthus', 'Kassadin', 'Katarina', 'Kayle', 'Kayn', 'Kennen', "Kha'Zix", 'Kindred', 'Kled', "Kog'Maw", 'LeBlanc', 'Lee Sin', 'Leona', 'Lillia', 'Lissandra', 'Lucian', 'Lulu', 'Lux', 'Malphite', 'Malzahar', 'Maokai', 'Master Yi', 'Miss Fortune', 'Wukong', 'Mordekaiser', 'Morgana', 'Nami', 'Nasus', 'Nautilus', 'Neeko', 'Nidalee', 'Nocturne', 'Nunu & Willump', 'Olaf', 'Orianna', 'Ornn', 'Pantheon', 'Poppy', 'Pyke', 'Qiyana', 'Quinn', 'Rakan', 'Rammus', "Rek'Sai", 'Rell', 'Renekt

There is much more information we can get about each champion from the API. It is a good idea to keep it all stored in one place to keep your data organized. An easy way to do this is to use a pandas dataframe.

In [210]:
import pandas as pd
from IPython.display import display, HTML
df = pd.DataFrame(champions, columns = ["Champion_object"])
display(df)

,Champion_object
0,"Champion(name='Aatrox', id=266, region='NA')"
1,"Champion(name='Ahri', id=103, region='NA')"
2,"Champion(name='Akali', id=84, region='NA')"
3,"Champion(name='Alistar', id=12, region='NA')"
4,"Champion(name='Amumu', id=32, region='NA')"
...,...
149,"Champion(name='Zed', id=238, region='NA')"
150,"Champion(name='Ziggs', id=115, region='NA')"
151,"Champion(name='Zilean', id=26, region='NA')"
152,"Champion(name='Zoe', id=142, region='NA')"


Now let's extract the stats for each champion:

In [211]:
names = []
armor = []
armor_per_level = []
attack_damage = []
attack_damage_per_level = []
attack_range = []
attack_speed = []
critical_strike_chance = []
critical_strike_chance_per_level = []
health = []
health_per_level = []
health_regen = []
health_regen_per_level = []
magic_resist = []
magic_resist_per_level = []
mana = []
mana_per_level = []
mana_regen = []
mana_regen_per_level = []
movespeed = []
percent_attack_speed_per_level = []
for i in champions:
    names.append(i.name)
    stats = i.stats
    armor.append(stats.armor)
    armor_per_level.append(stats.armor_per_level)
    attack_damage.append(stats.attack_damage)
    attack_damage_per_level.append(stats.attack_damage_per_level)
    attack_range.append(stats.attack_range)
    attack_speed.append(stats.attack_speed)
    critical_strike_chance.append(stats.critical_strike_chance)
    critical_strike_chance_per_level.append(stats.critical_strike_chance_per_level)
    health.append(stats.health)
    health_per_level.append(stats.health_per_level)
    health_regen.append(stats.health_regen)
    health_regen_per_level.append(stats.health_regen_per_level)
    magic_resist.append(stats.magic_resist)
    magic_resist_per_level.append(stats.magic_resist_per_level)
    mana.append(stats.mana)
    mana_per_level.append(stats.mana_per_level)
    mana_regen.append(stats.mana_regen)
    mana_regen_per_level.append(stats.mana_regen_per_level)
    movespeed.append(stats.movespeed)
    percent_attack_speed_per_level.append(stats.percent_attack_speed_per_level)
df["names"] = names
df["armor"] = armor
df["armor_per_level"] = armor_per_level
df["attack_damage"] = attack_damage
df["attack_damage_per_level"] = attack_damage_per_level
df["attack_range"] = attack_range
df["attack_speed"] = attack_speed
df["critical_strike_chance"] = critical_strike_chance
df["critical_strike_chance_per_level"] = critical_strike_chance_per_level
df["health"] = health
df["health_per_level"] = health_per_level
df["health_regen"] = health_regen
df["health_regen_per_level"] = health_regen_per_level
df["magic_resist"] = magic_resist
display(df)

,Champion_object,names,armor,armor_per_level,attack_damage,attack_damage_per_level,attack_range,attack_speed,critical_strike_chance,critical_strike_chance_per_level,health,health_per_level,health_regen,health_regen_per_level,magic_resist
0,"Champion(name='Aatrox', id=266, region='NA')",Aatrox,38,3.25,60.0,5.00,175,0.378558,0,0,580.0,90,3.0,1.00,32.0
1,"Champion(name='Ahri', id=103, region='NA')",Ahri,21,3.50,53.0,3.00,550,0.374700,0,0,526.0,92,5.5,0.60,30.0
2,"Champion(name='Akali', id=84, region='NA')",Akali,23,3.50,62.0,3.30,125,0.384615,0,0,500.0,105,8.0,0.50,37.0
3,"Champion(name='Alistar', id=12, region='NA')",Alistar,44,3.50,62.0,3.75,125,0.384615,0,0,600.0,106,8.5,0.85,32.0
4,"Champion(name='Amumu', id=32, region='NA')",Amumu,30,3.80,53.0,3.80,125,0.360023,0,0,615.0,80,9.0,0.85,32.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,"Champion(name='Zed', id=238, region='NA')",Zed,32,3.50,63.0,3.40,125,0.378558,0,0,584.0,85,7.0,0.65,32.0
150,"Champion(name='Ziggs', id=115, region='NA')",Ziggs,22,3.30,54.0,3.10,550,0.377415,0,0,536.0,92,6.5,0.60,30.0
151,"Champion(name='Zilean', id=26, region='NA')",Zilean,24,3.80,52.0,3.00,550,0.384615,0,0,504.0,82,5.5,0.50,30.0
152,"Champion(name='Zoe', id=142, region='NA')",Zoe,21,3.50,58.0,3.30,550,0.384615,0,0,560.0,92,6.5,0.60,30.0


As you can see the extraction step can be quite tedious but it'll make working with the data invaluably down the line. Let's get extract some items data before moving on.

In [212]:
attack = [] #how attack-oriented the champion is
defense = [] #how defense-oriented the champion is
magic = [] #how magic-oriented the champion is
items = [] 
for i in champions:
    attack.append(i.info.attack)
    defense.append(i.info.defense)
    magic.append(i.info.magic)
    items.append(i.recommended_itemsets)
df["attack"] = attack
df["defense"] = defense
df["magic"] = magic
df["items"] = items
display(df)

,Champion_object,names,armor,armor_per_level,attack_damage,attack_damage_per_level,attack_range,attack_speed,critical_strike_chance,critical_strike_chance_per_level,health,health_per_level,health_regen,health_regen_per_level,magic_resist,attack,defense,magic,items
0,"Champion(name='Aatrox', id=266, region='NA')",Aatrox,38,3.25,60.0,5.00,175,0.378558,0,0,580.0,90,3.0,1.00,32.0,8,4,3,"[{""<class 'cassiopeia.core.staticdata.champion..."
1,"Champion(name='Ahri', id=103, region='NA')",Ahri,21,3.50,53.0,3.00,550,0.374700,0,0,526.0,92,5.5,0.60,30.0,3,4,8,"[{""<class 'cassiopeia.core.staticdata.champion..."
2,"Champion(name='Akali', id=84, region='NA')",Akali,23,3.50,62.0,3.30,125,0.384615,0,0,500.0,105,8.0,0.50,37.0,5,3,8,"[{""<class 'cassiopeia.core.staticdata.champion..."
3,"Champion(name='Alistar', id=12, region='NA')",Alistar,44,3.50,62.0,3.75,125,0.384615,0,0,600.0,106,8.5,0.85,32.0,6,9,5,"[{""<class 'cassiopeia.core.staticdata.champion..."
4,"Champion(name='Amumu', id=32, region='NA')",Amumu,30,3.80,53.0,3.80,125,0.360023,0,0,615.0,80,9.0,0.85,32.0,2,6,8,"[{""<class 'cassiopeia.core.staticdata.champion..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,"Champion(name='Zed', id=238, region='NA')",Zed,32,3.50,63.0,3.40,125,0.378558,0,0,584.0,85,7.0,0.65,32.0,9,2,1,"[{""<class 'cassiopeia.core.staticdata.champion..."
150,"Champion(name='Ziggs', id=115, region='NA')",Ziggs,22,3.30,54.0,3.10,550,0.377415,0,0,536.0,92,6.5,0.60,30.0,2,4,9,"[{""<class 'cassiopeia.core.staticdata.champion..."
151,"Champion(name='Zilean', id=26, region='NA')",Zilean,24,3.80,52.0,3.00,550,0.384615,0,0,504.0,82,5.5,0.50,30.0,2,5,8,"[{""<class 'cassiopeia.core.staticdata.champion..."
152,"Champion(name='Zoe', id=142, region='NA')",Zoe,21,3.50,58.0,3.30,550,0.384615,0,0,560.0,92,6.5,0.60,30.0,1,7,8,"[{""<class 'cassiopeia.core.staticdata.champion..."


Before we can move on to transformation, there is one more vital step for extraction. APIs can often be buggy or outdated, so we cannot move on without some verification. We need to validate our data to make sure the API did not return anything incorrect or unexpected. We can do this simply by verifying the types of each column in our dataframe is what we expect them to be. It would also be fruitful to verify the values themselves however that requires extensive game knowledge that is not relevant to this tutorial.

In [213]:
df.dtypes

Champion_object                      object
names                                object
armor                                 int64
armor_per_level                     float64
attack_damage                       float64
attack_damage_per_level             float64
attack_range                          int64
attack_speed                        float64
critical_strike_chance                int64
critical_strike_chance_per_level      int64
health                              float64
health_per_level                      int64
health_regen                        float64
health_regen_per_level              float64
magic_resist                        float64
attack                                int64
defense                               int64
magic                                 int64
items                                object
dtype: object

Every type is as expected except for names. We expected string but got an object. Let's examine a bit further. 

In [214]:
for i in df["names"]:
    assert(isinstance(i, str))

Upon further inspection, the names columns seem to be correct. Now that we have extracted and validated our data, we are ready to move onto the next step in ETL, transformation.

# Transform
Now that we have all the data in one place, we want to make sure it is ready to be loaded into our data warehouse. There are many different ways we can approach this process. To name a few we could:
- Remove unnecessary columns
- Turn int64 columns into float64 so numerical columns are all a consistent type
- Derive new columns (e.g. combine health + health_per_level * 18 to get maximum health)
- Sorting by one of the stats instead of alphabetically
- Joining or grouping certain columns
- Splitting columns up
- Running new data validation checks after transformations

These are only a few of the things we can do in transformation as we can essentially prep our data however we please. Just as extraction, it is vital that we do transformation correctly so that the data is easy to use, clean, and consistent.

Luckily for us, using a wrapper instead of pulling straight from the API does a lot of the transformation already. Usually this step would involve json parsing but cass does it all for us. Let's start simple and get all of our numerical columns into one format.

In [215]:
for i in df:
    if df[i].dtypes == "int64":
        df[i] = df[i].astype("float64")
df.dtypes

Champion_object                      object
names                                object
armor                               float64
armor_per_level                     float64
attack_damage                       float64
attack_damage_per_level             float64
attack_range                        float64
attack_speed                        float64
critical_strike_chance              float64
critical_strike_chance_per_level    float64
health                              float64
health_per_level                    float64
health_regen                        float64
health_regen_per_level              float64
magic_resist                        float64
attack                              float64
defense                             float64
magic                               float64
items                                object
dtype: object

It's always a good idea to also inspect the values after any sort of transformation

In [216]:
display(df)

,Champion_object,names,armor,armor_per_level,attack_damage,attack_damage_per_level,attack_range,attack_speed,critical_strike_chance,critical_strike_chance_per_level,health,health_per_level,health_regen,health_regen_per_level,magic_resist,attack,defense,magic,items
0,"Champion(name='Aatrox', id=266, region='NA')",Aatrox,38.0,3.25,60.0,5.00,175.0,0.378558,0.0,0.0,580.0,90.0,3.0,1.00,32.0,8.0,4.0,3.0,"[{""<class 'cassiopeia.core.staticdata.champion..."
1,"Champion(name='Ahri', id=103, region='NA')",Ahri,21.0,3.50,53.0,3.00,550.0,0.374700,0.0,0.0,526.0,92.0,5.5,0.60,30.0,3.0,4.0,8.0,"[{""<class 'cassiopeia.core.staticdata.champion..."
2,"Champion(name='Akali', id=84, region='NA')",Akali,23.0,3.50,62.0,3.30,125.0,0.384615,0.0,0.0,500.0,105.0,8.0,0.50,37.0,5.0,3.0,8.0,"[{""<class 'cassiopeia.core.staticdata.champion..."
3,"Champion(name='Alistar', id=12, region='NA')",Alistar,44.0,3.50,62.0,3.75,125.0,0.384615,0.0,0.0,600.0,106.0,8.5,0.85,32.0,6.0,9.0,5.0,"[{""<class 'cassiopeia.core.staticdata.champion..."
4,"Champion(name='Amumu', id=32, region='NA')",Amumu,30.0,3.80,53.0,3.80,125.0,0.360023,0.0,0.0,615.0,80.0,9.0,0.85,32.0,2.0,6.0,8.0,"[{""<class 'cassiopeia.core.staticdata.champion..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,"Champion(name='Zed', id=238, region='NA')",Zed,32.0,3.50,63.0,3.40,125.0,0.378558,0.0,0.0,584.0,85.0,7.0,0.65,32.0,9.0,2.0,1.0,"[{""<class 'cassiopeia.core.staticdata.champion..."
150,"Champion(name='Ziggs', id=115, region='NA')",Ziggs,22.0,3.30,54.0,3.10,550.0,0.377415,0.0,0.0,536.0,92.0,6.5,0.60,30.0,2.0,4.0,9.0,"[{""<class 'cassiopeia.core.staticdata.champion..."
151,"Champion(name='Zilean', id=26, region='NA')",Zilean,24.0,3.80,52.0,3.00,550.0,0.384615,0.0,0.0,504.0,82.0,5.5,0.50,30.0,2.0,5.0,8.0,"[{""<class 'cassiopeia.core.staticdata.champion..."
152,"Champion(name='Zoe', id=142, region='NA')",Zoe,21.0,3.50,58.0,3.30,550.0,0.384615,0.0,0.0,560.0,92.0,6.5,0.60,30.0,1.0,7.0,8.0,"[{""<class 'cassiopeia.core.staticdata.champion..."


Upon closer inspection of the data, critical_strike_chance and critical_strike_chance_per_level are not useful columns as they are 0 across the board. Let's take them out.

In [217]:
df = df.drop(columns = ["critical_strike_chance","critical_strike_chance_per_level"])

We should also make the Champion_object column to a string to make it easier for loading.

In [218]:
df["Champion_object"] = df.astype("string")
for i in df["Champion_object"]:
    assert(isinstance(i, str))

Most of the columns look good now. However there is one more part which needs transforming. Our items column is a cass object which is not very useful in data analysis until we transform it into a more readable format. To do that we first need to understand what a recommended_itemset object contains using the champion [documentation](https://cassiopeia.readthedocs.io/en/latest/_modules/cassiopeia/core/staticdata/champion.html#Champion). 

In [219]:
for i in items[0:5]: #we loop through the array items instead of the pandas column so as to not make unintentional changes to our data
    for j in i: #only looping from 0 through 5 for demonstration purposes
        print(j.title)

AatroxARAM
AatroxFIRSTBLOOD
AatroxARAM
AatroxSL
AatroxSR
Beginner
AhriARAM
AhriFIRSTBLOOD
AhriARAM
AhriSL
AhriNPE
AhriNPE
AhriSR
Beginner
AkaliARAM
AkaliFIRSTBLOOD
AkaliARAM
AkaliSL
AkaliSR
Beginner
AlistarARAM
AlistarFIRSTBLOOD
AlistarARAM
AlistarNB
AlistarSR
Beginner
AmumuARAM
AmumuFIRSTBLOOD
AmumuARAM
AmumuSL
AmumuSR
Beginner


As you can see for each recommended item set, we have a few options. We are only interested in the *champion_name*SR one.

In [220]:
essential = dict()
for champion in items[0:5]:
    for itemsets in champion:
        if "SR" in itemsets.title:
            print(itemsets)

{"<class 'cassiopeia.core.staticdata.champion.RecommendedData'>": '<cassiopeia.core.staticdata.champion.RecommendedData object at 0x11a8efcd0>'}
{"<class 'cassiopeia.core.staticdata.champion.RecommendedData'>": '<cassiopeia.core.staticdata.champion.RecommendedData object at 0x11a8fa640>'}
{"<class 'cassiopeia.core.staticdata.champion.RecommendedData'>": '<cassiopeia.core.staticdata.champion.RecommendedData object at 0x11a9009a0>'}
{"<class 'cassiopeia.core.staticdata.champion.RecommendedData'>": '<cassiopeia.core.staticdata.champion.RecommendedData object at 0x11a90a970>'}
{"<class 'cassiopeia.core.staticdata.champion.RecommendedData'>": '<cassiopeia.core.staticdata.champion.RecommendedData object at 0x11a913ca0>'}


When parsing through wrapper or API defined objects, things can get a little bit hairy. Above we are given numerous RecommendedData objects. Let's do some more parsing to get what we want.

In [221]:
for champion in items[0:5]:
    for itemsets in champion:
        if "SR" in itemsets.title:
            for k in itemsets.item_sets:
                print(k.type)

starting
early
essential
starting
early
essential
starting
early
essential
starting
early
essential
starting
early
essential


We had to use cass defined methods called item_sets and type to get to where we are at. Now we are only interested in the essential set.

In [222]:
for champion in items[0:5]:
    for itemsets in champion:
        if "SR" in itemsets.title:
            for k in itemsets.item_sets:
                if k.type == "essential":
                    print(k, type(k))

{"<class 'cassiopeia.core.staticdata.champion.BlockData'>": '<cassiopeia.core.staticdata.champion.BlockData object at 0x11a8f1ac0>'} <class 'cassiopeia.core.staticdata.champion.ItemSet'>
{"<class 'cassiopeia.core.staticdata.champion.BlockData'>": '<cassiopeia.core.staticdata.champion.BlockData object at 0x11a8fafa0>'} <class 'cassiopeia.core.staticdata.champion.ItemSet'>
{"<class 'cassiopeia.core.staticdata.champion.BlockData'>": '<cassiopeia.core.staticdata.champion.BlockData object at 0x11a903790>'} <class 'cassiopeia.core.staticdata.champion.ItemSet'>
{"<class 'cassiopeia.core.staticdata.champion.BlockData'>": '<cassiopeia.core.staticdata.champion.BlockData object at 0x11a90da60>'} <class 'cassiopeia.core.staticdata.champion.ItemSet'>
{"<class 'cassiopeia.core.staticdata.champion.BlockData'>": '<cassiopeia.core.staticdata.champion.BlockData object at 0x11a916a90>'} <class 'cassiopeia.core.staticdata.champion.ItemSet'>


The docs tell us that the ItemSet object behaves like a dictionary, so we can finally get what we want.

In [223]:
essential = dict()
for champion in items[0:5]:
    for itemsets in champion:
        if "SR" in itemsets.title:
            for k in itemsets.item_sets:
                if k.type == "essential":
                    essential = k
    for i in list(essential.items.keys()):
        print(i.name)
    print("")


Trinity Force
Sterak's Gage
Blade of The Ruined King
Guardian Angel
Black Cleaver
Dead Man's Plate

Riftmaker
Demonic Embrace
Banshee's Veil
Rabadon's Deathcap
Void Staff
Cosmic Drive

Riftmaker
Demonic Embrace
Banshee's Veil
Rabadon's Deathcap
Void Staff
Cosmic Drive

Locket of the Iron Solari
Boots of Swiftness
Zeke's Convergence
Knight's Vow
Frozen Heart
Abyssal Mask

Sunfire Aegis
Mercury's Treads
Randuin's Omen
Spectre's Cowl
Warmog's Armor
Gargoyle Stoneplate



There we go! We were able to turn our recommended item column into our relevant item sets for each champion. Now let's load it up into our dataframe.

In [224]:
first,second,third,fourth,fifth,sixth = [],[],[],[],[],[]
essential = dict()
for champion in items:
    for itemsets in champion:
        if "SR" in itemsets.title:
            for k in itemsets.item_sets:
                if k.type == "essential":
                    essential = k

    item_names = list(essential.items.keys())
    first.append(item_names[0].id)
    second.append(item_names[1].id)
    try:
        third.append(item_names[2].id)
    except Exception as e:
        print(e)

list index out of range


Now we see a problem. The first two items have been successfully appended however the third item throws an error. Inconsistent data will be a common occurence in the transformation step, and it is our job to fix it. We expected there to be six items for each champion but it seems that some of them have less. We can fix this by raising exceptions.

In [225]:
first,second,third,fourth,fifth,sixth = [],[],[],[],[],[]
essential = dict()
count = 0
for champion in items:
    for itemsets in champion:
        if "SR" in itemsets.title:
            for k in itemsets.item_sets:
                if k.type == "essential":
                    essential = k

    item_names = list(essential.items.keys())
    try:
        first.append(item_names[0].id)
    except:
        print(champions[count])
    try:
        second.append(item_names[1].id)
    except:
        print(champions[count])
    try:
        third.append(item_names[2].id)
    except:
        print(champions[count])
        continue
    try:
        fourth.append(item_names[3].id)
    except:
        print(champions[count])
        continue
    try:
        fifth.append(item_names[4].id)
    except:
        print(champions[count])
        continue
    try:
        sixth.append(item_names[5].id)
    except:
        print(champions[count])
        continue
    count += 1

Champion(name='Jayce', id=126, region='NA')
Champion(name='Kennen', id=85, region='NA')
Champion(name='Lee Sin', id=64, region='NA')
Champion(name='Maokai', id=57, region='NA')
Champion(name='Nasus', id=75, region='NA')
Champion(name='Nocturne', id=56, region='NA')
Champion(name='Nunu & Willump', id=20, region='NA')
Champion(name='Nunu & Willump', id=20, region='NA')
Champion(name='Pantheon', id=80, region='NA')
Champion(name='Poppy', id=78, region='NA')
Champion(name='Pyke', id=555, region='NA')
Champion(name='Qiyana', id=246, region='NA')
Champion(name='Rakan', id=497, region='NA')
Champion(name='Rammus', id=33, region='NA')
Champion(name='Shaco', id=35, region='NA')
Champion(name='Urgot', id=6, region='NA')
Champion(name='Veigar', id=45, region='NA')


We have pinpointed exactly which champions have incomplete data using exceptions. Since *id* is an integer, we can just replace the empty item slots with *0*

In [226]:
first,second,third,fourth,fifth,sixth = [],[],[],[],[],[]
essential = dict()
count = 0
for champion in items:
    for itemsets in champion:
        if "SR" in itemsets.title:
            for k in itemsets.item_sets:
                if k.type == "essential":
                    essential = k

    item_names = list(essential.items.keys())
    try:
        first.append(item_names[0].id)
    except:
        first.append(0)
    try:
        second.append(item_names[1].id)
    except:
        second.append(0)
    try:
        third.append(item_names[2].id)
    except:
        third.append(0)
    try:
        fourth.append(item_names[3].id)
    except:
        fourth.append(0)
    try:
        fifth.append(item_names[4].id)
    except:
        fifth.append(0)
    try:
        sixth.append(item_names[5].id)
    except:
        sixth.append(0)

Let's verify that every list has the correct length before loading it into the dataframe

In [227]:
assert(len(first) == len(champions))
assert(len(second) == len(champions))
assert(len(third) == len(champions))
assert(len(fourth) == len(champions))
assert(len(fifth) == len(champions))
assert(len(sixth) == len(champions))

Everything looks good now. We have transformed our recommended item column into actual item IDs labeled first through sixth which will be much easier to use down the line. We can also remove the recommended items column from our dataframe since we have extracted the useful parts.

In [228]:
df["Item1"] = first
df["Item2"] = second
df["Item3"] = third
df["Item4"] = fourth
df["Item5"] = fifth
df["Item6"] = sixth
df = df.drop(columns=["items"])
display(df)

,Champion_object,names,armor,armor_per_level,attack_damage,attack_damage_per_level,attack_range,attack_speed,health,health_per_level,...,magic_resist,attack,defense,magic,Item1,Item2,Item3,Item4,Item5,Item6
0,"Champion(name='Aatrox', id=266, region='NA')",Aatrox,38.0,3.25,60.0,5.00,175.0,0.378558,580.0,90.0,...,32.0,8.0,4.0,3.0,3078,3053,3153,3026,3071,3742
1,"Champion(name='Ahri', id=103, region='NA')",Ahri,21.0,3.50,53.0,3.00,550.0,0.374700,526.0,92.0,...,30.0,3.0,4.0,8.0,4633,4637,3102,3089,3135,4629
2,"Champion(name='Akali', id=84, region='NA')",Akali,23.0,3.50,62.0,3.30,125.0,0.384615,500.0,105.0,...,37.0,5.0,3.0,8.0,4633,4637,3102,3089,3135,4629
3,"Champion(name='Alistar', id=12, region='NA')",Alistar,44.0,3.50,62.0,3.75,125.0,0.384615,600.0,106.0,...,32.0,6.0,9.0,5.0,3190,3009,3050,3109,3110,3001
4,"Champion(name='Amumu', id=32, region='NA')",Amumu,30.0,3.80,53.0,3.80,125.0,0.360023,615.0,80.0,...,32.0,2.0,6.0,8.0,3068,3111,3143,3211,3083,3193
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,"Champion(name='Zed', id=238, region='NA')",Zed,32.0,3.50,63.0,3.40,125.0,0.378558,584.0,85.0,...,32.0,9.0,2.0,1.0,3142,3111,3147,0,0,0
150,"Champion(name='Ziggs', id=115, region='NA')",Ziggs,22.0,3.30,54.0,3.10,550.0,0.377415,536.0,92.0,...,30.0,2.0,4.0,9.0,6655,3020,4637,3102,3089,3135
151,"Champion(name='Zilean', id=26, region='NA')",Zilean,24.0,3.80,52.0,3.00,550.0,0.384615,504.0,82.0,...,30.0,2.0,5.0,8.0,6655,3020,4637,3102,3089,3135
152,"Champion(name='Zoe', id=142, region='NA')",Zoe,21.0,3.50,58.0,3.30,550.0,0.384615,560.0,92.0,...,30.0,1.0,7.0,8.0,6655,3020,4637,3102,3089,3135


There is our final transformed dataframe. With all the data in an easy, readable format, we can confidently move on to the final step.

# Load
This step should be the most straightforward in the ETL pipeline. After properly extracting, validating, transforming, and validating again, the data should presumably be ready to simply be loaded into any sort of data warehouse and available for use. For our load we will be using sqlite3 since it is synergizes well with python and pandas.

In [229]:
import sqlite3
conn = sqlite3.connect("tutorial.db")
cursor = conn.cursor()
cursor.execute("DELETE FROM champions") #Remake table so as to not repeat values since this notebook will be run numerous times
cursor.execute("""
        CREATE TABLE IF NOT EXISTS champions (
        Champion_object TEXT,
        names TEXT,
        armor REAL,
        armor_per_level REAL,
        attack_damage REAL,
        attack_damage_per_level REAL,
        attack_range REAL,
        attack_speed REAL,
        health REAL,
        health_per_level REAL,
        health_regen REAL,
        health_regen_per_level REAL,
        magic_resist REAL,
        attack REAL,
        defense REAL,
        magic REAL,
        Item1 INTEGER,
        Item2 INTEGER,
        Item3 INTEGER,
        Item4 INTEGER,
        Item5 INTEGER,
        Item6 INTEGER
        );""")
temp = []
for i,j in df.iterrows():
        temp = (j["Champion_object"],
        j["names"], j["armor"], j["armor_per_level"],
        j["attack_damage"], j["attack_damage_per_level"],
        j["attack_range"], j["attack_speed"], j["health"],
        j["health_per_level"], j["health_regen"], j["health_regen_per_level"],
        j["magic_resist"], j["attack"],j["defense"],j["magic"],
        j["Item1"], j["Item2"], j["Item3"], j["Item4"], j["Item5"],
        j["Item6"])
        sml = "INSERT OR IGNORE INTO champions VALUES ("
        for ii in range(len(temp)):
            sml += "?,"
        sml = sml[:-1] + ")"
        cursor.execute(sml, temp)
conn.commit()
conn.close()

Let's make sure the table is loaded correctly

In [230]:
conn = sqlite3.connect("tutorial.db")
df_temp = pd.read_sql_query("SELECT * from champions", conn)
display(df_temp)

,Champion_object,names,armor,armor_per_level,attack_damage,attack_damage_per_level,attack_range,attack_speed,health,health_per_level,...,magic_resist,attack,defense,magic,Item1,Item2,Item3,Item4,Item5,Item6
0,"Champion(name='Aatrox', id=266, region='NA')",Aatrox,38.0,3.25,60.0,5.00,175.0,0.378558,580.0,90.0,...,32.0,8.0,4.0,3.0,3078,3053,3153,3026,3071,3742
1,"Champion(name='Ahri', id=103, region='NA')",Ahri,21.0,3.50,53.0,3.00,550.0,0.374700,526.0,92.0,...,30.0,3.0,4.0,8.0,4633,4637,3102,3089,3135,4629
2,"Champion(name='Akali', id=84, region='NA')",Akali,23.0,3.50,62.0,3.30,125.0,0.384615,500.0,105.0,...,37.0,5.0,3.0,8.0,4633,4637,3102,3089,3135,4629
3,"Champion(name='Alistar', id=12, region='NA')",Alistar,44.0,3.50,62.0,3.75,125.0,0.384615,600.0,106.0,...,32.0,6.0,9.0,5.0,3190,3009,3050,3109,3110,3001
4,"Champion(name='Amumu', id=32, region='NA')",Amumu,30.0,3.80,53.0,3.80,125.0,0.360023,615.0,80.0,...,32.0,2.0,6.0,8.0,3068,3111,3143,3211,3083,3193
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,"Champion(name='Zed', id=238, region='NA')",Zed,32.0,3.50,63.0,3.40,125.0,0.378558,584.0,85.0,...,32.0,9.0,2.0,1.0,3142,3111,3147,0,0,0
150,"Champion(name='Ziggs', id=115, region='NA')",Ziggs,22.0,3.30,54.0,3.10,550.0,0.377415,536.0,92.0,...,30.0,2.0,4.0,9.0,6655,3020,4637,3102,3089,3135
151,"Champion(name='Zilean', id=26, region='NA')",Zilean,24.0,3.80,52.0,3.00,550.0,0.384615,504.0,82.0,...,30.0,2.0,5.0,8.0,6655,3020,4637,3102,3089,3135
152,"Champion(name='Zoe', id=142, region='NA')",Zoe,21.0,3.50,58.0,3.30,550.0,0.384615,560.0,92.0,...,30.0,1.0,7.0,8.0,6655,3020,4637,3102,3089,3135


And we are done! The load step is complete and we have officially created a full ETL pipeline for the Riot League of Legends API. 

# Final thoughts
This tutorial showed you the basics of ETL. The tutorial mainly highlighted the process itself, the likely errors you would find along the way, and how to fix them. For some further readings and more detail on ETL, consider looking at the following links:
- ETL is not the only approach for creating a pipeline from data source -> data science task. Another popular approach is DAD which stands for Discover/Access/Distill: https://www.datasciencecentral.com/profiles/blogs/discover-access-distill-the-essence-of-data-science
- Cassiopeia docs: https://cassiopeia.readthedocs.io/en/latest/index.html
- Riot Games API: https://developer.riotgames.com/
- Further reading on ETL: https://hevodata.com/learn/steps-to-build-etl-pipeline/
- Further reading on ETL in python: https://www.xplenty.com/blog/building-an-etl-pipeline-in-python/
- Reading on ETL from multiple sources: https://www.xplenty.com/blog/get-data-from-multiple-sources/#:~:text=ETL%20